## Core concepts

- **Regression:** Regression predicts a continuous number. It gives real values as outcome.
- **Classification:** Classification predicts a categorical label. It gives a label as outcome.
- **Target Variable Analysis:** Here we examine the features of a target variable which is dependent in nature. It is necessary because once we have examined what our target variable is like then we can set the other data accordingly to get good results in analysis. It also involved transformation. This step profiles the target variable to align preprocessing, transformation, and modeling strategies for better results
- **Advanced data preprocessing:** It is like data processing, here we apply even more advanced strategies to pre process data.
- **Feature Engineering:** Here we create new features using the pre-existing ones to get better information or underlying information that our data holds.
- **Categorical Encoding:** When we have categorical values in our columns and we change it to numrical ones then it is known as categorical encoding.
- **Model Building:** It is the process of training and validating an algorithm with the use dependent and independent variables.
- **Model Evaluation:** Once our model is training then we test our model to evaluate the accuracy of our model, that is known as model evaluation.

## Importing libraries

- **pandas:** used to perform operations on structured data with tables
- **numpy:** used to perform mathematical operations on arrays, matrices 
- **matplotlib.pyplot:** used to plotting rows and columns in different kinds of charts (basic)
- **seaborn:** holds a collection of graphs and charts which can plot and visualize data in a better way (advanced) (built on matplotlib)
- **scipy.stats:** provides statistical functions, here **skew** measures asymmetry in data distribution
- **sklearn.model_selection:** **(train_test_split)** splits dataset into training and testing sets for model validation
- **sklearn.preprocessing:** **(StandardScaler, LabelEncoder)** StandardScaler normalizes numeric features to a normal scale, LabelEncoder converts categorical labels to numbers
- **sklearn.linear_model:** **LinearRegression** for linear regresion, linear regression is a machine learning algorithm based on the equation y = mx + c
**sklearn.metrics:** **(mean_squared_error, r2_score, mean_absolute_error)** these are metrics to evaluate a regression model's accuracy and error
**xgboost:** It's an efficient library for powerful machine learning models, often used for structured(tables) data prediction
**os:** provides operating system interaction capabilities.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb
import os

In [5]:
# setting plot style
sns.set_style('whitegrid')

## Data loading via kaggle

In [6]:
!pip install -q kaggle


[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Use Colab's file uploader
from google.colab import files
print("Please upload the kaggle.json file you downloaded from your Kaggle account.")
files.upload()

# Create a directory for the Kaggle API configuration
!mkdir -p ~/.kaggle
# Move the uploaded kaggle.json to the required directory
!cp kaggle.json ~/.kaggle/
# Set the correct permissions for the file
!chmod 600 ~/.kaggle/kaggle.json

print("\nKaggle API configured successfully.")

NameError: name 'files' is not defined